In [77]:
import pdfplumber, camelot, pprint
import fitz #type: ignore
import pandas as pd
import numpy as np
import json, os, re

In [3]:
path = os.path.join(os.getcwd(),"pdfs","1736224555418.pdf")

In [78]:
all_data = []
with pdfplumber.open(path) as doc:
    for i, page in enumerate(doc.pages):
        if i in [0, 1, 2]:
            text = page.extract_text()
            all_data.append(text)

In [79]:
pprint.pprint(all_data)

['SCHEME INFORMATION DOCUMENT\n'
 'QUANTUM LIQUID FUND\n'
 '(An Open-ended Liquid Scheme. A relatively low interest rate risk and '
 'relatively\n'
 'low cred it risk)\n'
 'Continuous Offer of Units at NAV Based Prices\n'
 'PRODUCT LABEL\n'
 'This product is suitable for investors Scheme Riskometer Benchmark '
 'Riskometer (Tier I)\n'
 'who are seeking*\n'
 '• Income over the short term\n'
 '• Investments in debt / money market\n'
 'instruments.\n'
 'Tier I Benchmark:\n'
 'CRISIL Liquid Debt A-I Index\n'
 'Investors understand that their\n'
 'principal will be at Low Risk\n'
 '* Investors should consult their financial advisers if in doubt about '
 'whether the product is suitable for them.\n'
 'Credit Risk → Relatively Low Moderate Relatively High\n'
 'Interest Rate Risk↓ (Class A) (Class B) (Class C)\n'
 'Relatively Low (Class I) A-I\n'
 'Moderate (Class II)\n'
 'Relatively High (Class III)\n'
 'MUTUAL FUND TRUSTEE COMPANY Asset Management Company\n'
 'Quantum Mutual Fund Quantum Tru

In [ ]:
scheme_tables = camelot.read_pdf(path, pages="4,5,6,7,8,9", flavor='lattice')
dfs = all_data = pd.concat([table.df for table in scheme_tables], ignore_index=True)
dfs = dfs.drop(dfs.columns[0], axis=1)
dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x) #applies to dataframe not series
dfs.iloc[:, 0] = dfs.iloc[:, 0].replace(r'^\s*$', np.nan, regex=True)
dfs.iloc[:, 0] = dfs.iloc[:, 0].ffill() #fill corresponding structure
new_column_names = ['Title'] + [f'Data{i}' for i in range(1, dfs.shape[1])]
dfs.columns = new_column_names
# html = dfs.to_html("check.html")
data_cols = [col for col in dfs.columns if col != 'Title']
result_dict = {}
for title, group_df in dfs.groupby('Title', sort=False):
    title = re.sub(r"\s+"," ",title).strip()
    title = "_".join(title.lower().split(" "))
    
    current_title_values = []
    for index, row in group_df.iterrows():
        for col_name in data_cols:
            cell_value = row[col_name]
            if isinstance(cell_value, str) and cell_value.strip() != '':
                current_title_values.append(cell_value)
    result_dict[title] = current_title_values

MANAGER

In [ ]:
manager_tables = camelot.read_pdf(path, pages="18", flavor='lattice')
dfs = all_data = pd.concat([table.df for table in manager_tables], ignore_index=True)
dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
fund_manager_data = {
    "fund_manager": [list(row) for _, row in dfs.iterrows()]
}

In [55]:
result_dict.update(fund_manager_data)
with open("final.json","w+") as data:
    json.dump(result_dict,data)